In [1]:
from kafka import KafkaProducer
import csv
from datetime import datetime
from uuid import uuid4
from cassandra.cluster import Cluster

# Initialisez votre client Kafka
producer = KafkaProducer(bootstrap_servers='localhost:9092')

# connect to Cassandra
cluster = Cluster(['localhost'])
session = cluster.connect()

# create keyspace and table
session.execute("CREATE KEYSPACE IF NOT EXISTS rss_data WITH replication = {'class': 'SimpleStrategy', 'replication_factor': '1'} AND durable_writes = true;")
session.set_keyspace('rss_data')
session.execute("CREATE TABLE IF NOT EXISTS lemonde_data (id uuid PRIMARY KEY, title text, link text, pubdate timestamp, description text);")

# prepare insert statement
prepared = session.prepare("INSERT INTO lemonde_data (id, title, link, pubdate, description) VALUES (?, ?, ?, ?, ?)")

# read data from CSV and insert into table
with open('lemonde_rss.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar='"')
    for row in reader:
        if row[0] == 'title':  # skip the header row
            continue
        id = uuid4()
        title = row[0]
        link = row[8]
        pubdate = datetime.strptime(row[2], '%a, %d %b %Y %H:%M:%S %z')
        description = row[4]
        session.execute(prepared, (id, title, link, pubdate, description))


In [2]:
# Run a query
rows = session.execute('SELECT * FROM lemonde_data')
for row in rows : 
    print(row.title)
    print(row.link)
    print("\n")

Maintien de l’ordre dans les autres pays d’Europe : qui contrôle les agissements de la police ?
https://www.lemonde.fr/les-decodeurs/article/2023/04/05/maintien-de-l-ordre-dans-les-autres-pays-d-europe-qui-controle-les-agissements-de-la-police_6168388_4355770.html


Avenir de la Nouvelle-Calédonie : le mouvement indépendantiste ouvert à des discussions avec le gouvernement
https://www.lemonde.fr/politique/article/2023/03/28/avenir-de-la-nouvelle-caledonie-le-mouvement-independantiste-ouvert-a-des-discussions-avec-le-gouvernement_6167349_823448.html


Guerre en Ukraine, en direct : le patron de l’AIEA attendu dans la matinée à la centrale nucléaire de Zaporijia
https://www.lemonde.fr/international/live/2023/03/29/guerre-en-ukraine-en-direct-plus-de-19-500-enfants-ukrainiens-deportes-illegalement-par-la-russie-selon-kiev_6167368_3210.html


Trottinette électrique : l’âge minimal d’utilisation va passer à 14 ans
https://www.lemonde.fr/politique/article/2023/03/29/trottinette-electrique-l-

In [8]:
#delete données
#session.execute('TRUNCATE lemonde_data')